In [2]:
import pandas as pd
import statsmodels
import numpy as np
import datetime
from datetime import date
from datetime import datetime
import time
import pandas as pd
import numpy as np
%matplotlib inline
import statistics
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from itertools import zip_longest

def driver_code():
    Capabilities = DesiredCapabilities.CHROME
    Capabilities["pageLoadStrategy"] = "normal"
    options = ChromeOptions()

    useragentarray = [
        "Mozilla/5.0 (Linux; Android 13) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.76 Mobile Safari/537.36"
    ]

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument(f"--user-data-dir=./profile{driver_num}")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    options.add_argument("disable-infobars")
    options.add_argument("disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(
        '/Users/paulconnolly/Desktop/ChromeDriver/chromedriver',
        options=options,
        desired_capabilities=Capabilities,
    )
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )

    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride", {"userAgent": useragentarray[0]}
    )
    driver.set_window_size(390, 844)
    options.add_argument("--disable-popup-blocking")
    #     driver.execute_script(
    #         """setTimeout(() => window.location.href="https://www.bet365.com.au", 100)"""
    #     )
    driver.get("https://www.flashscore.com/tennis/")
    time.sleep(3)
    return driver


def accept_cookies(driver):
    cookies = driver.find_elements(By.ID, "onetrust-accept-btn-handler")
    if(len(cookies) > 0):
        cookies[0].click()
    else:
        print("No Cookies to Click")

def get_participants(driver, element):
    element.click()
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    players = driver.find_elements(By.CSS_SELECTOR, ".participant__overflow")
    updated_players_list = []
    unique_players_list = []
    for player in players:
        if(player.text not in updated_players_list):
            updated_players_list.append(player.text)
            unique_players_list.append(player)
    return unique_players_list

def get_player_data(driver, players, dataframe):
    try:
        for player in players:
            player.click()
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[2])
            name = get_name(driver)
            first_name = name[0]
            last_name = name[1]
            age = get_age(driver)
            if(age == "N/A"):
                DOB = "N/A"
            else:
                DOB = age[1]
                age = age[0]
            player_id = get_player_id(driver)
            nationality = get_nationality(driver)
            new_row = {'Player ID': player_id,'First Name':first_name,'Last Name': last_name,'Age': age,'DOB': DOB,'Nationality':nationality}
            print(new_row)
            if(len(dataframe) > 1):
                last_row = dataframe.loc[len(dataframe) - 1]
                if(last_row.equals(new_row)):
                    continue
                else:
                    dataframe.loc[len(dataframe)] = new_row
                    driver.close()
                    driver.switch_to.window(driver.window_handles[1])
            else:       
                dataframe.loc[len(dataframe)] = new_row
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
    except:
        return
    
def player_data(driver):
    name = get_name(driver)
    first_name = name[0]
    last_name = name[1]
    age = get_age(driver)
    DOB = age[1]
    age = age[0]
    player_id = get_player_id(driver)
    nationality = get_nationality(driver)
    return first_name,last_name, age, DOB, player_id, nationality

def get_name(driver):
    name = driver.find_elements(By.CSS_SELECTOR, ".heading__name")
    first_name = ""
    last_name = ""
    for key in name:
        name = key.text
        name = name.split(" ")
        if(len(name) > 2):
            first_name = name[0]
            for x in range(1, len(name)):
                last_name += (name[x] + " ")
        else:
            first_name = name[0]
            last_name = name[1]
        return first_name, last_name
    
def get_age(driver):
    try:
        age = driver.find_elements(By.CSS_SELECTOR, ".jsl-age")
        for key in age:
            age = (key.text).split(" ")
            DOB = age[2].replace('(','').replace(')','')
            age = age[1]
        return age, DOB
    except:
        age = "N/A"
        return age
    
def get_player_id(driver):
    url = driver.current_url
    url = url.split("/")
    player_id = url[5]
    return player_id

def get_nationality(driver):
    try:
        nationality = driver.find_elements(By.CSS_SELECTOR, ".breadcrumb__text")
        if(nationality is None):
            nationality = "N/A"
        for key in nationality:
            nationality = key.text
        return nationality
    except:
        return " "

start = time.time()
today = date.today()
driver1 = driver_code()
d1 = today.strftime("%d_%m_%Y")
accept_cookies(driver1)
time.sleep(5)
my_columns = ['Player ID','First Name','Last Name','Age','DOB','Nationality']
final_dataframe = pd.DataFrame(columns = my_columns)
Parent_elements = driver1.find_elements(
                By.CSS_SELECTOR, ".event__match--twoLine")
players_list = []
for key in Parent_elements:
    try:
        time.sleep(2)
        driver1.execute_script("window.scrollBy(0,50)","")
        players = get_participants(driver1, key)
        get_player_data(driver1, players, final_dataframe)
        print("Players Written to Dataframe Successfully")
        driver1.close()
        driver1.switch_to.window(driver1.window_handles[0])
    except:
        driver1.close()
        driver1.switch_to.window(driver1.window_handles[0])
        continue
        
print("Scraper Completed")
driver1.quit()
final_dataframe

/var/folders/m2/30tqfzv54fx3fvq_xgkw697c0000gn/T/ipykernel_976/3867363139.py:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


{'Player ID': 'hMrlbuuQ', 'First Name': 'Damir', 'Last Name': 'Dzumhur', 'Age': '31', 'DOB': '20.05.1992', 'Nationality': 'BOSNIA AND HERZEGOVINA'}
{'Player ID': 'h6n0i9iE', 'First Name': 'Timofey', 'Last Name': 'Skatov', 'Age': '22', 'DOB': '21.01.2001', 'Nationality': 'KAZAKHSTAN'}
Players Written to Dataframe Successfully
{'Player ID': 'S6bHHtiI', 'First Name': 'Pablo', 'Last Name': 'Cuevas', 'Age': '38', 'DOB': '01.01.1986', 'Nationality': 'URUGUAY'}
{'Player ID': 'UkpY2sR5', 'First Name': 'Giulio', 'Last Name': 'Zeppieri', 'Age': '22', 'DOB': '07.12.2001', 'Nationality': 'ITALY'}
Players Written to Dataframe Successfully
{'Player ID': 'vD0zNRBM', 'First Name': 'Pierre-Hugues', 'Last Name': 'Herbert', 'Age': '32', 'DOB': '18.03.1991', 'Nationality': 'FRANCE'}
{'Player ID': 'CE7B5N90', 'First Name': 'Roman', 'Last Name': 'Andres Burruchaga ', 'Age': '21', 'DOB': '23.01.2002', 'Nationality': 'ARGENTINA'}
Players Written to Dataframe Successfully



KeyboardInterrupt



In [3]:
final_dataframe

,Player ID,First Name,Last Name,Age,DOB,Nationality
0,hMrlbuuQ,Damir,Dzumhur,31,20.05.1992,BOSNIA AND HERZEGOVINA
1,h6n0i9iE,Timofey,Skatov,22,21.01.2001,KAZAKHSTAN
2,S6bHHtiI,Pablo,Cuevas,38,01.01.1986,URUGUAY
3,UkpY2sR5,Giulio,Zeppieri,22,07.12.2001,ITALY
4,vD0zNRBM,Pierre-Hugues,Herbert,32,18.03.1991,FRANCE
5,CE7B5N90,Roman,Andres Burruchaga,21,23.01.2002,ARGENTINA
